# Related skill analysis

Using data scraped and parsed by SeekScraper.py, I'm now going to explore the data to see if I can find associations between different technologies and different job titles. The goal is to find out which technology I can learn next that will add the most value given my existing skillset.

For the purposes of this analysis, a programming language or technology will be referred to as a "skill".

Housekeeping Note: After the update of SeekScraper.py to search for jobs based on category rather than skill (e.g. 'data analyst' rather than 'python') and search multiple categories, I have changed the file names to differentiate between the original and updated dataset. The original datasets will be replaced with 'Job Details v1.csv' and 'Job Requirements v1.csv' to allow this code to still run as originally intended. The new datasets (used in later sections) will remain named 'Job Details.csv' and 'Job Requirements.csv'.

It is important to use set datasets for this analysis as the datasets generated by the scraper will change frequently as jobs are added and removed from the website. Additionally, scraping the datasets is quite a time-consuming process so it's best to save the results as an external file and work from there rather than re-scraping the data each time. Overall, this approach is more consistent and efficient when performing a demonstration or exploratory analysis, as I have below.

## Load data and libraries

Import the relevant libraries and data.

In [1]:
import pandas as pd
import numpy as np
import re

df_job_details = pd.read_csv('Job Details v1.csv')
df_job_requirements = pd.read_csv('Job Requirements v1.csv')

Initialise a list of possible technologies that may be found in the job advertisements (e.g. 'Python', 'R').

In [2]:
skills_list = ['sql', 'azure', 'python', 'linux', 'javascript', 'java', 'siebel', 'machine learning', 'docker', 
               'git', 'js', 'react', 'php', 'saas', 'html', 'sap', 'mysql', 'r', 'ruby', 'jenkins', 'c#', 'css', 
               'c++', 'c', 'aws', 'kubernetes', 'hadoop', 'vba', 'angular', '.net' ]

Note that the functions search by converting all text to lower-case, so the terms in skills_list can't use upper-case letters. Special characters are okay, but the terms will need to be given specific regex expressions to search by when the specific_regex dictionary is created (see below).

## Get a list of skills required for each job

First, I create a function that takes the job details (via the df_job_details and df_job_requirements dataframes) and a list of keywords (skills_list). It outputs a dataframe where each row 

In [3]:
def get_job_skills (df_job_details, df_job_requirements, skills_list, specific_regex):
    
    df_job_skills = pd.DataFrame(columns=(['Job title', 'URL'] + skills_list))
    df_job_requirements['Sentence'] = df_job_requirements['Sentence'].replace(np.nan, '')
    
    for url in df_job_details['URL'].tolist():
        
        # Get string of all job requirements, plus job title, in lowercase
        job_reqs = '. '.join(df_job_requirements[df_job_requirements['URL']==url]['Sentence'].tolist())
        job_title = df_job_details[df_job_details['URL']==url]['Job_Title'].tolist()[0]
        job_reqs = (job_title + ': ' + job_reqs).lower()
        
        # Initialise new row and skills_found
        new_row = [job_title, url]
        skills_found = False
        
        # Check each skill keyword
        for skill in skills_list:
            # Build a regex pattern to locate the keyword (use regex from specific_regex dictionary if present)
            if skill in specific_regex.keys():
                regex_pattern = specific_regex[skill]
            else:
                regex_pattern = '\\b(' + skill + ')\\b'
            
            # If the keyword is found, add a '1' to the new row of the dataframe, otherwise add '0'
            if re.search(regex_pattern, job_reqs) is not None:
                new_row.append(1)
                skills_found = True
            else:
                new_row.append(0)
                
        # Add the new row only if at least one skill keyword has been found
        if skills_found : df_job_skills.loc[df_job_skills.shape[0] + 1] = new_row
        
    return df_job_skills

I'm also going to initialize specific_regex, a dictionary which specifies regex expressions for skills which can be referred to by more than one name (e.g. 'AWS' vs 'Amazon Web Services') or contain unusual characters (e.g. 'C++', '.NET').

In [4]:
specific_regex = { 'c++' : '\\b(c\+\+)', 
                   'c' : '\\bc\\b[^\\+#]',
                   '.net' : '(\.net)\\b', 
                   'aws' : '\\b(aws|amazon web services)\\b',
                   'react' : '\\b(react|reactjs)\\b' }

The specific_regex dictionary is most important when special characters are used, as the default regex expression in the functions is created by surrounding the term by word boundary regex characters ('\\b') in order to prevent the functions from picking up programming languages are word fragments, particularly in the case of programming languages named for a single character, such as "C" or "R" (which would potentially be processed as appearing in almost every job description if the word boundary weren't used).

Additionally, adding a word boundary before/after a special character (e.g. '\\b\.net\\b' or '\\bc\+\+\\b') will cause issues as the word boundary in "C++" occurs after the "C" character, not after the final "+" character.

Using a specific_regex system means that most keywords will be treated normally, but allows for exceptions where required.

In [5]:
df_job_skills = get_job_skills(df_job_details, df_job_requirements, skills_list, specific_regex)
df_job_skills.head(5)

,Job title,URL,sql,azure,python,linux,javascript,java,siebel,machine learning,...,c#,css,c++,c,aws,kubernetes,hadoop,vba,angular,.net
1,Security Analyst (Level 2),https://www.seek.co.nz/job/50160948?type=stand...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Site Reliability Engineer,https://www.seek.co.nz/job/50160866?type=stand...,0,1,1,1,0,1,0,1,...,0,0,0,0,1,1,0,0,0,0
3,DevOps Support Engineer,https://www.seek.co.nz/job/50160716?type=stand...,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,C++ Software Engineer - 3D Rendering - $140k neg,https://www.seek.co.nz/job/50160715?type=stand...,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,Senior Developer,https://www.seek.co.nz/job/50160688?type=stand...,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


The number of rows in the outputted dataframe will tell me how many jobs were scraped and had at least one named skill listed in the job requirements.

In [6]:
df_job_skills.shape[0]

191

This means that there are 191 roles in the dataset which mention at least one of the listed skills.

## Get relevant roles

First, I'd like to see what jobs I currently have the listed skills for. I'm going to create a function that outputs the job skills dataframe filtered to contain only the jobs for which I have the requisite skills.

In [7]:
def get_relevant_roles(df, skills_list):
    df['Remove'] = 0
    for col_name in df:
        if col_name not in skills_list + ['URL', 'Job title', 'Remove']:
            df.loc[df[col_name]==1, ['Remove']] = 1
            
    df = df[df['Remove']==0]
    df = df.drop(['Remove'], axis=1)
    
    return df

In [8]:
relevant_roles = get_relevant_roles(df_job_skills, ['python', 'r', 'sql', 'vba'])
relevant_roles.head(5)

,Job title,URL,sql,azure,python,linux,javascript,java,siebel,machine learning,...,c#,css,c++,c,aws,kubernetes,hadoop,vba,angular,.net
9,Principal Analyst Pricing,https://www.seek.co.nz/job/50157450?type=stand...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,"Analytics Specialist Roles, AML Analytics",https://www.seek.co.nz/job/50156783?type=stand...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,"Manager Retail Analytics, Modelling",https://www.seek.co.nz/job/50154676?type=stand...,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,GIS Analyst,https://www.seek.co.nz/job/50138934?type=stand...,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,"BI Developer - x 2 Perm, Full-Time",https://www.seek.co.nz/job/50134090?type=stand...,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Once again, a count of the rows in the output dataframe will tell me how many roles I meet the requirements for.

In [9]:
relevant_roles.shape[0]

25

So, out of the 188 jobs in the dataset, I meet all of the programming language requirements for 26 of them with my current skillset. If I can only become proficient in one additional language, I want to maximise the number of roles for which I would now meet the requirements for.

## Impact of additional skills

I'm now going to create a function which outputs a dataframe showing how many jobs I would meet the requirements for if I learned one additional skill, depending on which skill I learned.

In [10]:
def get_new_skills_list(df_job_skills, skills_list, current_skills_list):
    
    # Initialise and populate output dataframe consisting of each skill name and the count of eligible roles
    df_new_skills = pd.DataFrame(columns=['Skill', 'Count'])
    for skill in skills_list:
        if skill not in current_skills_list:
            df_skill = get_relevant_roles(df_job_skills, current_skills_list + [skill])
            df_new_skills.loc[df_new_skills.shape[0] + 1] = [skill, df_skill.shape[0]]
    
    # Sort and return
    df_new_skills = df_new_skills.sort_values(by=['Count'], ascending=False)
    return (df_new_skills)

In [11]:
get_new_skills_list(df_job_skills, skills_list, ['python', 'r', 'sql', 'vba'])

,Skill,Count
20,c++,31
1,azure,30
6,machine learning,30
22,aws,29
4,java,28
8,git,27
26,.net,27
25,angular,26
11,php,26
24,hadoop,26


## Filtering out irrelevant roles

However, I'd like to try to narrow the roles down before performing the analysis. For example, I have two years of experience and so I'm probably not eligible for roles requesting more than 5 years' experience - my current role was requesting 4 years' experience and I got the job, so I'm expecting a little flexibility in this area, but I'll consider any advertisement requesting over 5 years' experience to be out of scope for me. Additionally, I don't have any post-graduate qualifications or management experience, so I'll probably rule out any manager/team lead roles or roles requesting a Masters or PhD. Finally, with only two years' experience, I'm firmly in the Junior/Intermediate range, and should probably filter out any "Senior" roles.

I've built a function which will analyse the job requirements for general job title categories (engineer, developer, analyst, etc.), as well as job titles referencing "data" (my area of interest) and those specifying team lead/manager/head in the title or mentioning postgraduate education in the description or job title.

In [12]:
def categorise_roles (df_job_details, df_job_requirements):

    # Initialise regex patterns for different categories and output dataframe
    regex_patterns = [ ['Lead', '(lead)\\b'],
                       ['Manager', '\\b(manager)\\b'],
                       ['Head', '\\b(head)\\b'],
                       ['Analyst', '\\b(analyst)\\b'],
                       ['Engineer', '\\b(engineer)\\b'],
                       ['Developer', '\\b(developer|dev)\\b'],
                       ['Junior', '\\b(junior|jnr|jr)\\b'],
                       ['Intermediate', '\\b(intermediate)\\b'],
                       ['Senior', '\\b(senior|snr|sr)\\b'],
                       ['Data', '\\b(data)\\b'],
                       ['Data Scientist', '\\b(data scientist)\\b'],
                     ]
    df_job_requirements['Sentence'] = df_job_requirements['Sentence'].replace(np.nan, '')
    df_out = pd.DataFrame(columns=['Job Title',
                                   'Years Experience', 'Lead', 'Manager', 'Head',
                                   'Analyst', 'Engineer', 'Developer',
                                   'Junior', 'Intermediate', 'Senior',
                                   'Data', 'Data Scientist', 'Postgraduate', 'URL'])
    
    for url in df_job_details['URL'].tolist():
        # Get string of all job requirements, plus job title, in lowercase
        job_reqs_list = df_job_requirements[df_job_requirements['URL']==url]['Sentence'].tolist()
        job_reqs = '. '.join(job_reqs_list)
        job_title = df_job_details[df_job_details['URL']==url]['Job_Title'].tolist()[0]
        job_reqs = (job_title + ': ' + job_reqs).lower()
        
        # Get years' experience required
        exp = 0
        for job_req in job_reqs_list:
            # Look for job_req where a number is followed by "years"
            job_req = job_req.lower()
            if re.search('\\b(\d+)\\+? years?', job_req) is not None:
                phrases = re.findall('(([^\.\n]*)\\b((\d+)-)?(\d+)\\+? years?([^\.\n]*))', job_req)
                for phrase in phrases:
                    # Look for key terms to indicate that the phrase refers to the length of 
                    # experience required rather than the age of the company
                    if re.search('\\b(experience|hands on|development|\d+\\+ years?)\\b', phrase[0]) is not None:
                        # Where a range is given, take the lower
                        years = re.findall('\\b(\d+)?-?(\d+)\\+? years?', phrase[0])
                        for years_set in years:
                            exp_new = int(years_set[0]) if years_set[0] != '' else int(years_set[1])
                            if exp_new > exp : exp = exp_new
        
        # Initialise new row and convert job_title to lowercase for regex analysis
        new_row = [job_title, exp]
        job_title = job_title.lower()
        
        # Scan job titles for key terms and add information to new row
        for [col_name, regexp] in regex_patterns:
            if re.search(regexp, job_reqs) is not None:
                new_row.append(1)
            else:
                new_row.append(0)
                
        # Check for terms in job description that may indicate that postgraduate study is required
        if re.search('\\b(phd|masters|post(-| )?graduate|doctorate)\\b', job_reqs) is not None:
            new_row.append(1)
        else:
            new_row.append(0)
        
        # Add new row to dataframe
        new_row.append(url)
        df_out.loc[df_out.shape[0] + 1] = new_row
        
    return df_out

The "years experience" part of this task was a bit more complicated than anticipated as many companies liked to mention how long they'd been in business. I got around this by checking for other keywords in the surrounding phrase (e.g. 'experience', 'hands-on', 'development') which indicated that they were specifying length of experience rather than talking about how long the company had been operating. This seemed to work for the initial sample that I was working with, although there might be some inconsistencies in a larger sample size.

I will now generate the categorised roles dataframe and use it to eliminate irrelevant roles - those requiring more than 5 years' experience and those requiring postgraduate study, as well as any team lead or similar roles and roles that are considered to be at "Senior" level.

In [13]:
df_categorised_roles = categorise_roles(df_job_details, df_job_requirements)
exclude_urls = df_categorised_roles['URL'][df_categorised_roles['Postgraduate']==1].tolist() + \
               df_categorised_roles['URL'][df_categorised_roles['Years Experience']>5].tolist() + \
               df_categorised_roles['URL'][df_categorised_roles['Senior']==1].tolist() + \
               df_categorised_roles['URL'][df_categorised_roles['Lead']==1].tolist() + \
               df_categorised_roles['URL'][df_categorised_roles['Manager']==1].tolist() + \
               df_categorised_roles['URL'][df_categorised_roles['Head']==1].tolist()

I'm now going to edit the get_job_skills function to include an exclude_urls parameter, which will allow me to retrieve a job skills dataframe with the specified URLs filtered out.

In [14]:
def get_job_skills (df_job_details, df_job_requirements, skills_list, specific_regex, exclude_urls=[]):
    
    df_job_skills = pd.DataFrame(columns=(['Job title', 'URL'] + skills_list))
    df_job_requirements['Sentence'] = df_job_requirements['Sentence'].replace(np.nan, '')
    
    for url in df_job_details['URL'].tolist():
        if url not in exclude_urls:
        
            # Get string of all job requirements, plus job title, in lowercase
            job_reqs = '. '.join(df_job_requirements[df_job_requirements['URL']==url]['Sentence'].tolist())
            job_title = df_job_details[df_job_details['URL']==url]['Job_Title'].tolist()[0]
            job_reqs = (job_title + ': ' + job_reqs).lower()

            # Initialise new row and skills_found
            new_row = [job_title, url]
            skills_found = False

            # Check each skill keyword
            for skill in skills_list:
                # Build a regex pattern to locate the keyword (use regex from specific_regex dictionary if present)
                if skill in specific_regex.keys():
                    regex_pattern = specific_regex[skill]
                else:
                    regex_pattern = '\\b(' + skill + ')\\b'

                # If the keyword is found, add a '1' to the new row of the dataframe, otherwise add '0'
                if re.search(regex_pattern, job_reqs) is not None:
                    new_row.append(1)
                    skills_found = True
                else:
                    new_row.append(0)

            # Add the new row only if at least one skill keyword has been found
            if skills_found : df_job_skills.loc[df_job_skills.shape[0] + 1] = new_row
        
    return df_job_skills

I'll now use this edited function to get an updated version of the previous dataframes with the irrelevant roles filtered out and see how many I'm qualified for.

In [15]:
df_job_skills_filtered = get_job_skills(df_job_details, df_job_requirements, skills_list, 
                                        specific_regex, exclude_urls)
relevant_roles_filtered = get_relevant_roles(df_job_skills_filtered, ['python', 'r', 'sql', 'vba'])
relevant_roles_filtered.shape[0]

9

Now I'll look at the exact job titles to see what kind of roles I'm likely to be applying to.

In [16]:
relevant_roles_filtered['Job title'].tolist()

['GIS Analyst',
 'Data Scientist',
 'Data Scientist',
 'Data Analyst',
 'Data Analyst',
 'Junior Data Engineer',
 'Financial Data Analyst',
 'Junior Data Engineer',
 'Data Engineer']

Just glancing at the job titles, this list seems more or less what I'd expect - mostly data analyst and data engineer roles with a few data scientist roles sprinkled in. I didn't specify "data" in any of my filtering or parameters, but given a Python/R/SQL skillset I'd be expecting mostly data-focused roles.

Now I'll see how many of the roles I would meet the criteria for if I learned one additional skill.

In [17]:
get_new_skills_list(df_job_skills_filtered, skills_list, ['python', 'r', 'sql', 'vba'])

,Skill,Count
6,machine learning,12
22,aws,12
20,c++,12
1,azure,11
4,java,11
8,git,11
11,php,10
25,angular,10
24,hadoop,10
16,ruby,10


## Skill total count

As a final addition, I will create a function which will return a full count of the number of times each skill appears in the role requirements list, regardless of which other skills also appear in the same set of role requirements.

In [18]:
def get_total_skills_list(df_job_skills, skills_list):
    
    # Initialise dataframe and add each skill with the number of jobs that it is required for
    df_skills = pd.DataFrame(columns=['Skill', 'Count'])
    for skill in skills_list:
        skill_total = sum(df_job_skills[skill].tolist())
        df_skills.loc[df_skills.shape[0] + 1] = [skill, skill_total]
      
    # Sort and return
    df_skills = df_skills.sort_values(by=['Count'], ascending=False)
    return df_skills

# Updated dataset

Overall, I've managed to develop a functionalised approach to finding relevant job titles for a skillset and/or list of parameters. The functions and inputs can easily be adapted for different skill sets or keyword requirements by adding terms to the relevant lists or dictionaries.

I have now updated the SeekScraper.py script so that it searches for roles based on job titles (such as "data analyst") rather than skills (such as "python") and so that it is able to search through multiple terms and compile the resulting job advertisements into a single dataset.

This should give me a larger dataset to demonstrate how the functions developed above can be used to see which skills are most in-demand for a collection of jobs advertised on Seek. It will also show how the tool I've developed can be used both ways - first, by examining which roles are relevant to an existing skillset and, second, by examing which skills are most relevant to a group of similar roles. Both applications are useful when job hunting and when investigating which new skills to focus on.

1) Retrieve and parse the data (using SeekScraper.py), then import the resulting datasets into the current analysis script.

Note: I am now using the more recent dataset, rather than the version 1 (v1) set.

In this iteration, I searched for jobs with the terms "data analyst", "data engineer", and "data scientist".

In [19]:
import pandas as pd
import numpy as np
import re

df_job_details = pd.read_csv('Job Details.csv')
df_job_requirements = pd.read_csv('Job Requirements.csv')

2) Create a list of possible required skills. If applicable (for skills containing non-alphanumeric characters or which are known by more than one name), create regex expressions to search for the relevant keywords.

In [20]:
skills_list = ['sql', 'azure', 'python', 'linux', 'javascript', 'java', 'siebel', 'machine learning', 'docker', 
               'git', 'js', 'react', 'php', 'saas', 'html', 'sap', 'mysql', 'r', 'ruby', 'jenkins', 'c#', 'css', 
               'c++', 'c', 'aws', 'kubernetes', 'hadoop', 'vba', 'angular', '.net' ]
specific_regex = { 'c++' : '\\b(c\+\+)', 
                   'c' : '\\bc\\b[^\\+#]',
                   '.net' : '(\.net)\\b', 
                   'aws' : '\\b(aws|amazon web services)\\b',
                   'react' : '\\b(react|reactjs)\\b' }

  3)   Categorise the roles by keywords (such as "data", "engineer", or "manager") and create a list of roles to be excluded from the analysis.

In this case, I will be excluding roles which do not have "data" in the job title. Even though the search terms meant that information was only scraped from roles which mentioned "data analyst", "data scientist", etc., these terms could have appeared in the role description and I only want to focus on role with "data" in the job title.

In [21]:
df_categorised_roles = categorise_roles(df_job_details, df_job_requirements)
exclude_urls = df_categorised_roles['URL'][df_categorised_roles['Data']==0].tolist()

4) Get a dataframe showing the skills required for each role, excluding those specified by exclude_urls.

In [24]:
df_job_skills = get_job_skills(df_job_details, df_job_requirements, skills_list, specific_regex, exclude_urls)
df_job_skills.shape[0]

76

From here, we can see that there are 76 role descriptions where the job title included the word "data" and where at least one of the listed skills was mentioned in the job requirements.

(5) Generate a dataframe showing which skills are most in-demand for the set criteria.

To see which skills would be most useful given an existing skillset, use get_new_skills_list.
To see which skills are most useful overall, use get_total_skills_list.

In [23]:
df_skill_count = get_total_skills_list(df_job_skills, skills_list)
df_skill_count

,Skill,Count
1,sql,44
3,python,40
25,aws,28
2,azure,19
18,r,17
9,docker,16
6,java,15
4,linux,13
8,machine learning,13
10,git,10


Based on this, it looks like by far the most in-demand skills for data roles are Python and SQL, with AWS a distant second.

# Conclusion

Overall, this analysis demonstrates how the functions and scripts developed above and in SeekScraper.py can be used to search jobs in two different ways:
- Looking at the skills a person has and presenting relevant roles, as well as suggesting a new skill that could be learned to maximise eligible roles
- Looking at an area of interest a person has and presenting skills that are often required in related roles

I'm hoping that this tool will be useful in my future job hunts, as well as to better direct my learning within the field of data analysis.